In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from keras_facenet import FaceNet

# Path to your local dataset
data_dir = r'C:/Deeplearning/genenv/Data science/Deep Learning/project 1/Avinash'

# Load the dataset from the directory (resize and batch the images)
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),  # Resize images to a uniform size
    batch_size=32,          # Define batch size
    label_mode='int'        # Specify label type (int, categorical, etc.)
)

# Augmentation layers to introduce variability in the images, including normalization
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

# Apply augmentation to the training dataset
augmented_train_dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y))

# Load the FaceNet model
embedder = FaceNet()

# Function to detect faces in an image
def detect_faces(image):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    return faces

# Function to extract and resize face
def extract_face(image):
    faces = detect_faces(image)
    for (x, y, w, h) in faces:
        face = image[y:y+h, x:x+w]  # Crop the face
        face_resized = cv2.resize(face, (160, 160))  # Resize to 160x160 for FaceNet
        return face_resized
    return None

# Function to get embeddings
def get_embeddings(face_image):
    embeddings = embedder.embeddings([face_image])
    return embeddings[0]

# Initialize lists for embeddings and names
known_face_embeddings = []
known_face_names = []

# Loop through each person's folder
for person_name in os.listdir(data_dir):
    person_dir = os.path.join(data_dir, person_name)
    if os.path.isdir(person_dir):
        # Process each image in the person's folder
        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            image = cv2.imread(image_path)
            if image is not None:
                face = extract_face(image)
                if face is not None:
                    embedding = get_embeddings(face)
                    known_face_embeddings.append(embedding)
                    known_face_names.append(person_name)

# Convert lists to numpy arrays
known_face_embeddings = np.array(known_face_embeddings)
known_face_names = np.array(known_face_names)

# Print to check
print(known_face_embeddings)
print(known_face_names)

# Initialize the webcam (0 is the default camera)
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise Exception("Could not open video device")

while True:
    ret, frame = cap.read()  # Capture frame-by-frame
    if not ret:
        break
    face = extract_face(frame)
    if face is not None:
        embedding = get_embeddings(face)
        name = "Unknown"
        # Compare the embeddings to known faces
        for known_embedding, known_name in zip(known_face_embeddings, known_face_names):
            distance = np.linalg.norm(embedding - known_embedding)
            if distance < 0.6:  # Threshold for face recognition
                name = known_name
                break
        # Draw the name on the frame
        cv2.putText(frame, name, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        # Draw rectangle on the face
        for (x, y, w, h) in detect_faces(frame):
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

    # Display the frame
    cv2.imshow('Video', frame)
    
    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


Found 115 files belonging to 6 classes.

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s